In [ ]:
from model import layers, activations, networks, utils, optimizers, losses, metrics, paddings, initializations
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import random

### Prepare data
Keras is used to fetch the data

In [ ]:
(x_train , y_train), (x_test , y_test) = keras.datasets.mnist.load_data()
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

Normalization

In [ ]:
x_train = x_train / 255
x_test = x_test / 255

Reshape the arrays to fit the models' dimension requirement

In [ ]:
x_train = utils.expand_dims(x_train, 4)
y_train = utils.expand_dims(y_train, 2)
x_test = utils.expand_dims(x_test, 4)
y_test = utils.expand_dims(y_test, 2)

print (f'{x_train.shape=}')
print (f'{y_train.shape=}')
print (f'{x_test.shape=}')
print (f'{y_test.shape=}')

### Build the neural network structre

In [ ]:
model = networks.FeedForward([
    layers.Input(input_shape=(28, 28, 1)),
    layers.Convolution(nr_kernels=4, kernel_size=(3, 3), activation=activations.ReLu),
    layers.MaxPooling((2, 2)),
    layers.Flatten(),
    layers.Dense(nr_neurons=64, activation=activations.Sigmoid),
    layers.Dense(nr_neurons=10, activation=activations.Softmax),
    layers.Output()
])

The network is compiled to internally connect it's layers and initialize the model.

In [ ]:
model.compile(
    optimizer=optimizers.adam(),
    loss=losses.categorical_crossentropy,
    metric=metrics.Accuracy
)

In [ ]:
model.summary()

### Train the model

In [ ]:
model.train(x_train, y_train, epochs=30, batch_size=500)

In [ ]:
model.plot_loss()

### Evaluate the model

In [ ]:
model.evaluate(x_test[:500], y_test[:500])

### Explore the inner workings
Pick a random image from the testing dataset.

In [ ]:
i = random.randint(0, len(x_test))
image = x_test[i]
plt.imshow(image, cmap='gray')

Use it to predict a number and show the probability distribution of the outcome.

In [ ]:
p = model.predict(image)
plt.bar(np.arange(0, 10), p.reshape(10,))

Every layer of the model can be accessed to explore their output. Here we iterate over all the kernels of the convolutional layer to explore what they learned to focus on in images.

In [ ]:
for i in range(model.layers[1].k):
    a = model.layers[1].o[:, :, i]
    plt.imshow(a, cmap='gray')
    plt.xlabel(f'kernel {str(i)}')
    plt.show()